In [1]:
# Cell 1
import cudf
import pandas as pd
import gc

<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cuda module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.driver module instead.


In [2]:
# Cell 2
# Load transactions
df = cudf.read_csv(
    '../data/input_data/transactions_train.csv',
    usecols=['t_dat', 'customer_id', 'article_id'],
    dtype={'t_dat': 'string', 'customer_id': 'string', 'article_id': 'int32'},
)
df['t_dat'] = cudf.to_datetime(df['t_dat'])
last_ts = df['t_dat'].max()
cut_ts = last_ts - pd.Timedelta(days=7)

# Use last 21 days of history for popularity
pop_start = cut_ts - pd.Timedelta(days=12)
df = df[(df['t_dat'] > pop_start) & (df['t_dat'] <= cut_ts)]

In [3]:
# Cell 3
# Calculate time-decayed popularity
# Day distance from cut_ts
df['diff_days'] = (cut_ts - df['t_dat']).dt.days
df['pop_score'] = 1.0 / (df['diff_days'] + 1)

# Aggregate per article
pop_items = (
    df.groupby('article_id')['pop_score']
      .sum()
      .reset_index()
      .sort_values('pop_score', ascending=False)
      .head(24)  # Top 24 items
)
pop_items['value'] = pop_items['pop_score'].astype('float32')
pop_items['window_type'] = 'global_pop'
pop_items = pop_items.drop('pop_score', axis=1)

In [4]:
# Cell 4
# Assign these items to ALL customers
# Load sample_submission to get all customer_ids
sub = cudf.read_csv(
    '../data/input_data/sample_submission.csv',
    usecols=['customer_id'],
    dtype={'customer_id': 'string'}
)
sub['customer_id'] = sub['customer_id'].str[-16:].str.hex_to_int().astype('int64')

# Cross join (Cartesian product)
# Since pop_items is small (24 rows), this is efficient
sub['key'] = 1
pop_items['key'] = 1
candidates = sub.merge(pop_items, on='key').drop('key', axis=1)

In [5]:
# Cell 5
# Save
candidates = candidates.to_pandas()
# Map back to hex if needed, but your pipeline seems to handle int/hex mix in build_candidates
# We will save with int customer_id to match your other parquets
candidates.to_parquet('../data/outputs/candidates_popularity.parquet', index=False)

del df, sub, pop_items, candidates
gc.collect()

0